In [9]:
include("../src/Dyn3d.jl")
import Dyn3d: ConfigBody, ConfigJoint, SingleBody, SingleJoint, 
              System, Motions, Dof, NumParams, Soln,
              AddBody, AddJoint, AssembleSystem!, InitSystem!, HERK!

using BenchmarkTools, Compat

INFO: Precompiling module BenchmarkTools.


In [2]:
# choose config files 
include("../src/Config_files/2dLinkobj.jl")

In [3]:
# add bodys
bodys = Vector{SingleBody}(nbody) # body system
for i = 1:nbody
    bodys[i] = AddBody(i, config_body) # add body
end

# add joints
joints = Vector{SingleJoint}(njoint) # joint system
for i = 1:njoint
    joints[i] = AddJoint(i, config_joints[i]) # add joint
end

# assemble system to a chain
system = System(ndim, nbody, njoint, gravity, num_params)
bodys, joints, system = AssembleSystem!(bodys, joints, system)
system

ndim = 2, njoint = 8, nbody = 8
ndof = 48, nudof = 8, ncdof = 40, np = 7, na = 1
udof = [3, 9, 15, 21, 27, 33, 39, 45]
udof_p = [9, 15, 21, 27, 33, 39, 45]
udof_a = [3]
nudof_HERK = 7, ncdof_HERK = 41
udof_HERK = [9, 15, 21, 27, 33, 39, 45]
gravity = [0.0, 0.0, 0.0]
kinmap = [1 1]


In [4]:
# # test function UpdatePosition!
# bodys, joints, system = UpdatePosition!(bodys, joints, system)

# # test function UpdateVelocity!
# for i = 1:nbody
#     @assert length(bodys[i].v) == 6
#     @assert length(bodys[i].Xp_to_b) == 36
#     @assert length(joints[i].vJ) == 6
# end
# v = ones(Float64,24)
# bodys, joints, system, vJ = UpdateVelocity!(bodys, joints, system, v)


# do some output
# bodys[1]
# bodys[2]
# joints[1]
system

ndim = 2, njoint = 8, nbody = 8
ndof = 48, nudof = 8, ncdof = 40, np = 7, na = 1
udof = [3, 9, 15, 21, 27, 33, 39, 45]
udof_p = [9, 15, 21, 27, 33, 39, 45]
udof_a = [3]
nudof_HERK = 7, ncdof_HERK = 41
udof_HERK = [9, 15, 21, 27, 33, 39, 45]
gravity = [0.0, 0.0, 0.0]
kinmap = [1 1]


In [5]:
# init system
bodys, joints, system, soln = InitSystem!(bodys, joints, system)
solns = (Soln)[]
push!(solns, soln)

1-element Array{Dyn3d.ConstructSystem.Soln,1}:
 Dyn3d.ConstructSystem.Soln{Float64}(0.0, 0.001, [0.0, 0.0, 0.785398, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], Float64[], Float64[])

In [6]:
# advance in time
idx = 0
@time begin
while soln.t < tf
    soln, bodys, joints, system = HERK!(soln, bodys, joints, system)
    push!(solns, soln)
    idx += 1
    if mod(idx,500) == 1 @printf("itr = %d, t = %.3f, dt = %e\n", idx, soln.t, soln.dt) end
end
end

itr = 1, t = 0.001, dt = 4.810511e-05
itr = 301, t = 0.031, dt = 1.185335e-04
itr = 601, t = 0.067, dt = 1.184831e-04
itr = 901, t = 0.102, dt = 1.185315e-04
itr = 1201, t = 0.138, dt = 1.186946e-04
itr = 1501, t = 0.174, dt = 1.190095e-04
itr = 1801, t = 0.209, dt = 1.194530e-04
itr = 2101, t = 0.245, dt = 1.199602e-04
itr = 2401, t = 0.281, dt = 1.204585e-04
itr = 2701, t = 0.318, dt = 1.208801e-04
itr = 3001, t = 0.354, dt = 1.211611e-04
itr = 3301, t = 0.390, dt = 1.212460e-04
itr = 3601, t = 0.427, dt = 1.211015e-04
itr = 3901, t = 0.463, dt = 1.207293e-04
itr = 4201, t = 0.499, dt = 1.201717e-04
itr = 4501, t = 0.535, dt = 1.195145e-04
itr = 4801, t = 0.571, dt = 1.188966e-04
itr = 5101, t = 0.606, dt = 1.185137e-04
itr = 5401, t = 0.642, dt = 1.185312e-04
itr = 5701, t = 0.678, dt = 1.189132e-04
itr = 6001, t = 0.713, dt = 1.195161e-04
itr = 6301, t = 0.749, dt = 1.202684e-04
itr = 6601, t = 0.785, dt = 1.211233e-04
itr = 6901, t = 0.822, dt = 1.220071e-04
itr = 7201, t = 0.859,

In [7]:
# solns[end].t
solns[end].qJ
solns[end].v
solns[end].v̇
solns[end].λ

41-element Array{Float64,1}:
 -0.0122689 
  0.00255893
  0.00554699
  0.00450463
  0.0244582 
  0.0       
 -0.011203  
 -0.00182002
 -0.00394707
  0.0218659 
  0.0       
 -0.00752907
 -0.00355383
  ⋮         
 -0.00348716
  0.0       
  0.00244941
  0.00159814
  0.00319678
 -0.00493285
  0.0       
  0.00193459
  0.00135646
  0.00271292
 -0.00386917
  0.0       

In [10]:
@benchmark HERK!(soln, bodys, joints, system)

BenchmarkTools.Trial: 
  memory estimate:  1.49 MiB
  allocs estimate:  5601
  --------------
  minimum time:     2.488 ms (0.00% GC)
  median time:      2.589 ms (0.00% GC)
  mean time:        2.959 ms (4.38% GC)
  maximum time:     30.140 ms (7.42% GC)
  --------------
  samples:          1685
  evals/sample:     1

In [ ]:
@benchmark HERK!(soln, bodys, joints, system)